## BoZ - Exchange Rate Data

#### Importing all necessary libraries.

In [9]:
import csv
import os
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from urllib.parse import urljoin

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

import pandas as pd

## Fetch data from BoZ website
#### Delete the first 3 rows.
#### Data retrieved from table and saved to .CSV file. The code below creates a new first column that is populated with the current date.


In [10]:
def fetch_and_store_data(url, table_id):
    response = requests.get(url)     # Send a GET request to the URL

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser') # Parse the HTML content
        
        table = soup.find('table', {'id': table_id}) #Finding the specific table by ID
        
        if table:
            # Get current date
            current_date = datetime.now().strftime('%Y-%m-%d')
            
            output_file = f'data/data.csv'

            # Open a CSV file to store the data
            with open(output_file, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                

                row_count = 0
                for row in table.find_all('tr'):
                    # Skip the first thee (3) rows that contain headers
                    if row_count < 3:
                        row_count += 1
                        continue
                    
                    # Add current date to each row and Extract data from each cell in the row
                    row_data = [current_date] + [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
                    
                    # Write to .CSV file
                    writer.writerow(row_data)
            print(f"Data has been fetched from table '{table_id}' and stored in '{output_file}'")
        else:
            print(f"Table with ID '{table_id}' not found on the page.")
    else:
        print("Failed to retrieve the webpage.")

#URL and  Table IDs
url = 'https://www.boz.zm/index.htm'
table_id = 'bank_rates_desktop'

fetch_and_store_data(url, table_id)

Data has been fetched from table 'bank_rates_desktop' and stored in 'data/data.csv'


## Append .CSV Files
#### Appending .CSV files to create a large dataset over time.

In [11]:
def append_csv_files(folder_x, folder_y, output_file):
    # List all CSV files in folder X
    csv_files = [file for file in os.listdir(folder_x) if file.endswith('.csv')]
    
    # Initialize an empty DataFrame to store appended data
    appended_data = pd.DataFrame()
    
    # Iterate through CSV files in folder X
    for file in csv_files:
        # Read each CSV file without header
        data = pd.read_csv(os.path.join(folder_x, file), header=None)
        
        # Append data vertically to the DataFrame
        appended_data = pd.concat([appended_data, data], ignore_index=False)
    
    # Check if the output directory exists, if not, create it
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    # Write the appended data to the output CSV file in folder Y
    appended_data.to_csv(output_file, index=False, header=True, mode='a')

# Example usage
folder_x =  '../project/raw_data'
folder_y = '../project/cleaned_data/'
output_file = os.path.join(folder_y,'final_data.csv')

append_csv_files(folder_x, folder_y, output_file)
print(f"Data has appended'{output_file}'")

Data has appended'/Users/bouffmbm/Documents/CC-TM-MM/project/final_data/final_data_test.csv'
